In [ ]:
%load_ext autoreload
%autoreload 2
import sys,os
sys.path.append('../')   
import numpy as np
from model.erroneousPreference import erroneousPreference
from utility import  paramz
from kernel import RBF
from numpy.linalg import norm

from inference import laplace_generic as LAP
from bayesopt import create_comparison_data, BayesOpt


In [ ]:
# We have two variables bounded between 0.1-20 and 1-8
k_bounds, d_bounds = (0.1, 20.), (1., 8.)
bounds = np.vstack([k_bounds, d_bounds])
# True values are 10 and 2
k_true, d_true = 10., 2.
true_vector = [k_true, d_true]
# We will try to find the true values with Bayesian optimization preference learning


In [ ]:
# This creates data of 5 comparisons to initalize the GP.
# When creating this data, we assume the user always selects the alternative that has
# closer L-2 distance to the true_vector
n_samp = 5
data = {}
data["X"] , data["Pairs"] = create_comparison_data(bounds, true_vector, n_samp)

# data["X"] contains the vector values (k and d) for each compared alternative
# data["Pairs"] shows the pair wise comparisons. The values here shows the index of X compared. The index showed first is the one preferred/selected
# For example if it shows [7,4] it means X[7] and X[4] are compared and X[7] were preferred.

In [ ]:
# This is our GP model

# define kernel and hyperparams
Kernel = RBF

#kernel parameter dictionary
params={'lengthscale': {'value':1.5*np.ones(data["X"].shape[1], float), 
                        'range': bounds,
                        'transform': paramz.logexp()},
        'variance': {'value':np.array([1.0]), 
                        'range':np.vstack([[0.01, 10.01]]),
                        'transform': paramz.logexp()}#variance not used in exactPreference
          }


# Ignore the ones below

#model.optimize_hyperparams(num_restarts=2)
#model.sample(nsamples=10000, tune=5000)

# Optimize parameters using Laplace Method 

Below is our 10 iterations for pairwise learning.

- `infer` object optimizes the hyperparameters of GP for the data and we also use it to create predictions with GP
- `BayesOpt` class proposes next X to try with BayesOpt using Expected Improvement Acquisition function (see inside this class in `bayesopt/bayesopt.py` to learn more)
- We choose the option that has the smaller l2 distance to true_vector
- Then we update the `infer` class with the new comparison added to the data

In [ ]:
# Iteratively find the optimal point
n_comparisons = 10

model = erroneousPreference(data,Kernel,params)
infer = LAP.inference_laplace(data, 
                              RBF, 
                              model.params,
                              model._log_likelihood,
                              model._grad_loglikelihood,
                              model._hess_loglikelihood)

infer.optimize(recompute_grad_hessian_at_params_change=True,num_restarts=2)
bo = BayesOpt(infer, bounds)

for i in range(n_comparisons):
    new_data = bo.propose_next(10,5).flatten()
    current_max_X = bo.x_max
    current_max_ind = bo.x_max_ind
    
    left_pref = norm(new_data - np.array(true_vector)) < norm(current_max_X - np.array(true_vector))    
    print(f"Current max: {current_max_X}; Compared: {new_data}")
    
    Pairs, X = data["Pairs"], data["X"]
    
    X = np.vstack([X, new_data])
    print(X.shape)
    new_ind = len(X) - 1
    preferred_index = np.array([new_ind, current_max_ind]) if left_pref else np.array([current_max_ind, new_ind])
    
    print(f"Preferred: {X[preferred_index[0]]}")
    
    Pairs = np.vstack([Pairs, preferred_index])
    data = {}
    data["Pairs"], data["X"] = Pairs, X

    model = erroneousPreference(data,Kernel,params)
    infer = LAP.inference_laplace(data, 
                              RBF, 
                              model.params,
                              model._log_likelihood,
                              model._grad_loglikelihood,
                              model._hess_loglikelihood)

    infer.optimize(recompute_grad_hessian_at_params_change=True,num_restarts=2)
    bo = BayesOpt(infer, bounds)

print(f"Optimal point: {bo.x_max}")